Instalación de las dependencias necesarias

In [ ]:
!pip install transformers datasets
!pip install wandb
!pip install trl==0.11.3

Preparación del conjunto de datos

In [ ]:
import random

def generate_lots_of_strings(n=10):
    list_of_strings = []
    for _ in range(n):
        # Crea un número aleatorio entre 10 y 50 y múltiplo de tres
        numero_aleatorio = random.randint(10, 50)
        # while numero_aleatorio % 3 != 0:
        #     numero_aleatorio = random.randint(10, 50)
        s = '{"text": "<s>[INST] %d es igual a [/INST]</s>"}' % (numero_aleatorio)
        list_of_strings.append(s)

    # Save the list of strings to a file
    with open("drive/MyDrive/list_of_strings.jsonl", "w") as file:
        file.write("\n".join(list_of_strings))

generate_lots_of_strings(200)

In [ ]:
def build_dataset(
    config,
    dataset_name='/content/drive/MyDrive/list_of_strings.jsonl',
    input_min_text_length=2,
    input_max_text_length=100,
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
    tokenizer.pad_token = tokenizer.eos_token
    print('--')
    # load imdb with datasets
    # ds = load_dataset(dataset_name, split="train")
    ds = load_dataset('json', data_files='/content/drive/MyDrive/list_of_strings.jsonl', split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    print('---2')

    # input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        # sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["input_ids"] = tokenizer.encode(sample["review"])
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
from datasets import load_dataset

# Reemplaza 'ruta/a/tu/dataset' con la ruta a tu archivo de datos
# dataset = build_dataset(config)
ds = load_dataset('json', data_files='/content/drive/MyDrive/list_of_strings.jsonl', split="train")
tokenizador.pad_token = tokenizador.eos_token

def tokenize_function(examples):
    return tokenizador(examples['text'], padding='max_length', truncation=True, max_length=64)

tokenized_train_dataset = ds.map(tokenize_function, batched=True, remove_columns=['text'])
tokenized_train_dataset.set_format(type="torch")

assert len(tokenized_train_dataset) > 0, "the dataset is empty"

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 200
})

Definición de la función de recompensa

In [ ]:
def comprobar_suma(cadena):
    # Diccionario para convertir números en texto a números en entero
    cadena = cadena.replace('[INST]','')
    texto_a_numero = {
        "uno": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9
    }

    # Extraer el número inicial en decimal
    numero_inicial = int(cadena.split()[0])

    # Extraer los números en texto y sumarlos
    palabras = cadena.split()[3:]  # Ignorar "es igual a"
    suma_numeros = sum(texto_a_numero[palabra] for palabra in palabras if palabra in texto_a_numero)

    # Calcular la diferencia
    diferencia = numero_inicial - suma_numeros

    # Devolver 0 si la suma es exacta, o la diferencia si no lo es
    return abs(diferencia)

Configuración del modelo y el tekonizador

In [ ]:
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead, create_reference_model

model_name = "lvwerra/gpt2-imdb"  # Puedes reemplazarlo con el modelo que prefieras
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ref_model = create_reference_model(model)
tokenizador = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
import wandb

wandb.init(
    project='LLM_PPO_training',
    name=model_name + '_autoregressive_instruct_fine_tuning'
)

In [ ]:
def collator(data):
  return dict((key, [d[key] for d in data]) for key in data[0])

In [ ]:
def reward(prompt, output):
    # Implementa aquí la lógica para evaluar la respuesta
    # Debe devolver un valor numérico que represente la calidad de la respuesta
    return torch.zeros_like(prompt)

Configuración del PPOTrainer

In [ ]:
from trl import PPOTrainer, PPOConfig

# Configuración de PPO
config = PPOConfig(
    model_name=model,
    learning_rate=3e-5,
    batch_size=4,
    forward_batch_size=2,
    log_with='wandb'
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:212: UserWarning: Note that using `forward_batch_size` is deprecated, use `mini_batch_size` instead. By setting it you overwrite `mini_batch_size` which affects both the batch size during forward passes and also the mini batch size for PPO optimization.
  warnings.warn(


In [ ]:
# Inicialización del PPOTrainer
ppo_trainer = PPOTrainer(
    model=model,
    ref_model=ref_model,
    config=config,
    dataset=tokenized_train_dataset,
    tokenizer=tokenizador,
    data_collator=collator
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [ ]:
print(tokenized_train_dataset[0])

{'input_ids': tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,    27,    82,
        36937, 38604,    60,  1160,  1658, 45329,   723,   257, 46581, 38604,
           60,  3556,    82,    29]), 'attention_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


Entrenamiento del modelo

In [ ]:
from tqdm import tqdm

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "pad_token_id": tokenizador.eos_token_id,
    "temperature": 0.01
}

In [ ]:
for epoch, batch in enumerate(tqdm(ppo_trainer.dataloader)):
  query_tensors = batch["input_ids"]

  response_tensors = []
  for query in query_tensors:
    # gen_len = output_length_sampler()
    # generation_kwargs["max_new_tokens"] = gen_len
    generation_kwargs["max_new_tokens"] = 100
    query_response = ppo_trainer.generate(query, **generation_kwargs).squeeze()
    response_len = len(query_response) - len(query)
    response_tensors.append(query_response[-response_len:])
  batch["response"] = [tokenizador.decode(r.squeeze()) for r in response_tensors]

  texts = [print(f"Q: {q}\nR: {r}") for q, r in zip(batch["input_ids"], batch["response"])]
  # pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
  # positive_scores = [
  #     item["score"]
  #     for output in pipe_outputs
  #     for item in output
  #     if item["label"] == "POSITIVE"
  # ]
  # rewards = [torch.tensor(scocre) for score in positive_scores]
  rewards = [torch.tensor([
      reward(inp, res)
      for inp, res in zip(batch["input_ids"], batch["response"])
  ])]

  stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
  ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/50 [00:04<?, ?it/s]

Q: tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,    27,    82,
        36937, 38604,    60,  1367,  1658, 45329,   723,   257, 46581, 38604,
           60,  3556,    82,    29], device='cuda:0')
R: <br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><
Q: tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256,

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
import torch

In [ ]:
query_tensors = torch.tensor(tokenized_train_dataset["input_ids"])
query_tensors = [query_tensor for query_tensor in query_tensors]
response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [ ]:
answer = tokenizador.decode(response_tensors[0], skip_special_tokens=True)
print(answer)

<s>[INST] 20 es igual a [/INST]</s>


In [ ]:
entrenador.train()

AttributeError: 'PPOTrainer' object has no attribute 'train'

Guardado del modelo fine-tuneado

In [ ]:
modelo.save_pretrained('mod_modelo')
tokenizador.save_pretrained('tok_modelo')